## Longitudinal Car Following Models

In [2]:
import math

In [4]:
def my_tanh(x):
    
    if x > 50:
        return 1
    elif x < -50:
        return -1
    else:
        return (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)

In [1]:
def formd(x):
    return "{:.2f}".format(float(x))

### IDM (Intelligent Driver Model)

In [1]:
class IDM:
    
    """
    class for the Intelligent Driver Model (IDM) 
    
    Attributes:
        v0: Desired speed in meters per second (m/s).
        T : Desired time gap in seconds (s).
        s0 : Minimum gap in meters (m).
        a : Maximum acceleration in meters per second squared (m/s^2).
        b : Comfortable deceleration in meters per second squared (m/s^2).
        speedlimit: Speed limit affecting the IDM (initially set to 1000, indicating no restriction).
        bmax : Maximum possible deceleration value.
    """

    def __init__(self, v0, T, s0, a, b):
        """
        Initializes an IDM instance with the provided parameters.

        Parameters:
            v0: Desired speed in meters per second (m/s).
            T: Desired time gap in seconds (s).
            s0: Minimum gap in meters (m).
            a: Maximum acceleration in meters per second squared (m/s^2).
            b: Comfortable deceleration in meters per second squared (m/s^2).
        """
        self.v0 = v0
        self.T = T
        self.s0 = s0
        self.a = a
        self.b = b
        self.speedlimit = 1000  # initially set to 1000, indicating no restriction
        self.bmax = 9  # maximum possible deceleration value


    def copy(self, long_model):
        """
        Copies the attributes from another IDM instance (long_model) to this instance.

        Parameters:
            long_model (IDM): The IDM instance from which to copy the attributes.
        """
        self.v0 = long_model.v0
        self.T = long_model.T
        self.s0 = long_model.s0
        self.a = long_model.a
        self.b = long_model.b
        self.speedlimit = long_model.speedlimit
        self.bmax = long_model.bmax
        

    def calc_acc_free(self, v):
        """
        Calculate the free acceleration for the IDM.

        Parameters:
            v : speed of the subject vehicle in meters per second (m/s).

        Returns:
            acc_free : The calculated free acceleration in meters per second squared (m/s^2).
        """
        v0eff = max(0.01, min(self.v0, self.speedlimit))
        
        if v < v0eff:
            acc_free = self.a * (1 - (v / v0eff) ** 4)
        else:
            acc_free = self.a * (1 - v / v0eff)
        return acc_free


    
    def calc_acc_int(self, s, v, vl, al):
        """
        Calculate the IDM interaction acceleration.

        Parameters:
            s : The actual gap to the leading vehicle in meters (m).
            v : The actual speed of the vehicle in meters per second (m/s).
            vl : The speed of the leading vehicle in meters per second (m/s).
            al : The acceleration of the leading vehicle in meters per second squared (m/s^2).
                      (This parameter is ignored in the current IDM implementation.)

        Returns:
            act_int: The calculated interaction acceleration in meters per second squared (m/s^2).
        """
        sstar = self.s0 + max(0, v * self.T + 0.5 * v * (v - vl) / math.sqrt(self.a * self.b))
        acc_int = -self.a * (sstar / max(s, 0.1 * self.s0)) ** 2
        
        return max(-self.bmax, acc_int)


### Adaptive Cruise Control (ACC) Model 

ACC: Has same parameters as IDM but exactly triangular steady state and "cooler" reactions if gap too small

The "triangular" aspect could symbolize the three points of action - accelerating to reach a target speed, decelerating to maintain a safe distance from the vehicle ahead, and cruising at a steady speed once the desired distance and speed are achieved. 




**for the ACC acceleration module**

Desired Gap (sstar): The desired gap sstar is calculated using the IDM formula, considering the current speed of the vehicle, the speed difference between the following and leading vehicle, and the IDM parameters. This gap represents the distance the vehicle aims to maintain from the leading vehicle under current conditions.

Free Acceleration (accFree): The calcAccFree method is called to calculate the free acceleration, which is the acceleration the vehicle would adopt in the absence of a leading vehicle influencing its behavior.

IDM Acceleration (accIntIDM): The IDM interaction acceleration accIntIDM is calculated based on the desired gap sstar and the actual gap s, applying the IDM formula. This acceleration is then added to the free acceleration to get accIDM.

CAH (Collision Avoidance Heuristic) Acceleration (accCAH): This part of the code calculates an acceleration based on a heuristic approach to avoid collisions, considering the vehicles' speeds and the actual gap.

Combining IDM and CAH Accelerations (accMix): The accMix variable combines the IDM acceleration and the CAH acceleration. The combination is based on whether the IDM acceleration is greater than the CAH acceleration. If not, a weighted mix is calculated, factoring in the vehicle's deceleration capability.

Final Acceleration (accACC): The final acceleration accACC is a mix of accMix and accIDM, weighted by the cool factor, which represents how "cool" or cautious the ACC system is when the gap is too small.

In [1]:
class ACC:
    
    def __init__(self, v0, T, s0, a, b):
        """
        Initializes an ACC instance with the parameters.

        Parameters:
            v0 : Desired speed in meters per second (m/s).
            T : Desired time gap in seconds (s).
            s0 : Minimum gap in meters (m).
            a: Maximum acceleration in meters per second squared (m/s^2).
            b: Comfortable deceleration in meters per second squared (m/s^2).
        """
        self.v0 = v0
        self.T = T
        self.s0 = s0
        self.a = a
        self.b = b
        self.cool = 0.99  # cooler reactions if the gap is too small
        self.speedlimit = 1000  # effective speed limit
        self.bmax = 9  # maximum deceleration

    def copy(self, long_model):
        """
        Copies the attributes from another ACC instance (long_model) to this instance.

        Parameters:
            long_model (ACC): The ACC instance from which to copy the attributes.
        """
        self.v0 = long_model.v0
        self.T = long_model.T
        self.s0 = long_model.s0
        self.a = long_model.a
        self.b = long_model.b
        self.cool = long_model.cool  # Assume cool factor should be copied as well
        self.speedlimit = long_model.speedlimit
        self.bmax = long_model.bmax

    def calc_acc_free(self, v):
        """
        Calculate the free acceleration for the ACC, which is the same as for IDM.

        Parameters:
            v (float): The actual speed of the vehicle in meters per second (m/s).

        Returns:
            acc_free: The calculated free acceleration in meters per second squared (m/s^2).
        """
        v0eff = max(0.01, min(self.v0, self.speedlimit))
        if v < v0eff:
            acc_free = self.a * (1 - (v / v0eff) ** 4)
        else:
            acc_free = self.a * (1 - v / v0eff)
        return acc_free

    def calc_acc_int(self, s, v, vl, al=0):
        """
        Calculate the ACC interaction acceleration.

        Parameters:
            s : The actual gap to the leading vehicle in meters (m).
            v : The actual speed of the vehicle in meters per second (m/s).
            vl: The speed of the leading vehicle in meters per second (m/s).
            al: The acceleration of the leading vehicle in meters per second squared (m/s^2), defaults to 0.

        Returns:
            float: The calculated interaction acceleration in meters per second squared (m/s^2).
        """

        # Determine the effective desired speed
        v0eff = min(self.v0, self.speedlimit)

        # Calculate the desired gap
        sstar = self.s0 + max(0, v * self.T + 0.5 * v * (v - vl) / math.sqrt(self.a * self.b))
        
        # Calculate the free acceleration
        acc_free = self.calc_acc_free(v)

        # Calculate the IDM interaction acceleration
        acc_int_idm = -self.a * (sstar / max(s, 0.1 * self.s0)) ** 2
        acc_idm = acc_free + acc_int_idm

        # Calculate the CAH (Collision Avoidance Heuristic) acceleration
        acc_cah = (v * v * al / (vl * vl - 2 * s * al)) if (vl * (v - vl) < -2 * s * al) else al - (v - vl) ** 2 / (2 * max(s, 0.1)) * (1 if v > vl else 0)
        acc_cah = min(acc_cah, self.a)

        # Combine IDM and CAH accelerations
        acc_mix = acc_idm if acc_idm > acc_cah else acc_cah + self.b * math.tanh((acc_idm - acc_cah) / self.b)

        # Calculate the final ACC acceleration
        acc_acc = self.cool * acc_mix + (1 - self.cool) * acc_idm
        acc_int = max(-self.bmax, acc_acc - acc_free)

        return acc_int

### Mixed Traffic Model 

**Notes:** Placeholder variables for global constants (glob_acc_lat_b_max, glob_acc_lat_b_ref, etc.) are used, assuming these are defined elsewhere in your Python code 

In [1]:
class MTM:
    
        def __init__(self, long_model, s0y, s0y_lat, s0y_b, s0y_lat_b, sens_lat, tau_lat_ovm, sens_dvy):
            """
            Initializes an MTM instance with the specified parameters.
    
            Parameters:
                long_model: The underlying longitudinal car-following model (e.g., ACC).
                s0y (float): Lateral attenuation scale for longitudinal veh-veh interaction [m].
                s0y_lat (float): Lateral attenuation scale for lateral veh-veh interaction [m].
                s0y_b (float): Lateral attenuation scale for longitudinal boundary-veh interaction [m].
                s0y_lat_b (float): Lateral attenuation scale for lateral boundary-veh interaction [m].
                sens_lat (float): Sensitivity (desired lateral speed)/(longitudinal accel) [s].
                tau_lat_ovm (float): Time constant for lateral OVM [s].
                sens_dvy (float): Sensitivity of lateral relative speed, similar to FVDM [s/m].
            """
            self.long_model = long_model
            self.s0y = s0y
            self.s0y_b = s0y_b
            self.s0y_lat = s0y_lat
            self.s0y_lat_b = s0y_lat_b
            self.sens_lat = sens_lat     # this part is somewhat different from the equation, the given original explnation 
            self.tau_lat_ovm = tau_lat_ovm
            self.sens_dvy = sens_dvy
    
            # Assuming global constants for boundary parameters are defined elsewhere
            self.acc_lat_int_max = 4 * long_model.b  # Example of linking to the long_model's attribute
            self.acc_lat_b_max = glob_acc_lat_b_max  # Placeholder for global constants
            self.acc_lat_b_ref = glob_acc_lat_b_ref
            self.acc_long_b_ref = glob_acc_long_b_ref
            self.antic_factor_b = glob_antic_factor_b
            self.nj = 8  # Number of discrete steps

        def copy(self, mixed_model):
            """
            Creates a deep copy of the provided MTM instance.
    
            Parameters:
                mixed_model (MTM): The MTM instance to copy.
            """
            # Assuming long_model has its own copy method
            self.long_model = mixed_model.long_model.copy()
            self.s0y = mixed_model.s0y
            self.s0y_b = mixed_model.s0y_b
            self.s0y_lat = mixed_model.s0y_lat
            self.s0y_lat_b = mixed_model.s0y_lat_b
            self.sens_lat = mixed_model.sens_lat
            self.tau_lat_ovm = mixed_model.tau_lat_ovm
            self.sens_dvy = mixed_model.sens_dvy


        ##############################################################
        ## Longitudinal Acceleration 
        ##############################################################

        def calc_acc_long_int(dx, dy, vx, vxl, axl, Ll, Wavg):
            
            """
            Calculate the interaction acceleration for longitudinal direction 
            
            Parameters:
                dx = distance between the leader and follower for horizontal
                dy = distance between the leader and follower for the vertical
                vx = speed of subject vehicle 
                vxl = speed of the leader vehicle 
                Ll = length of the leader 
                Wavg = width average of leader and subject
                long_par_reduct_factor = additional factor needed to impliment the alpha for paralel situation 
        
               
            returns:  veh veh longitudinal acceleration 
            
            """
        
            sx = max(0, dx - Ll)
            sy = abs(dy) - Wavg
        
            acc_cf_int = self.long_model.calc_acc_int(sx, vx, vxl, axl)
            alpha = min(exp(-sy/self.s0y),1)
        
            #if ((dx < Ll)&& (sy> 0)):
            #    alpha = alpha*long_par_reduct_factor
        
            return alpha*acc_cf_int


        def calc_acc_leader_select(dx, dy, vx, vxl, axl, Ll, Wavg):
        
            '''
            Simplified calcAccLongInt to select leaders and followers
            NOTE: Differences to calcAccLongInt:
                  (i) take the maximum of long and lat attenuation because I select
                       partners just on long acceleration!
                  (ii) no special provision for laterally neighboring vehs:
                       global.longParReductFactor
                       (for selecting neighbors, just assume full long force)
            '''
        
            """
            Calculate the interaction acceleration for longitudinal direction 
            
            Parameters:
                dx = distance between the leader and follower for horizontal
                dy = distance between the leader and follower for the vertical
                vx = speed of subject vehicle 
                vxl = speed of the leader vehicle 
                Ll = length of the leader 
                Wavg = width average of leader and subject
               
            returns:  veh veh longitudinal acceleration 
            
            """
        
            sx = max(0, dx - Ll)
            sy = abs(dy) - Wavg
            s0ylarger = max(this.s0y, s0ylarger)
        
            acc_cf_int = self.long_model.calc_acc_int(sx, vx, vxl, axl)
            alpha = min(exp(-sy/self.s0ylarger),1)
        
            return alpha*acc_cf_int
    
        ##############################################################
        ## Lateral Acceleration 
        ##############################################################

        def calc_acc_lat_free(vy): 
            
                """
                Calculate the free acceleration for lateral direction 
                
                Parameters:
                    vy: lateral speed of the subject vehicle 
                    tau_lat_OVM : speed adaptation time 
        
                returns: free lateral acceleration 
                
                """
            
                return -vy/self.tau_lat_OVM        

        def calc_acc_lat_int(x, xl, y, yl, vx, vxl, vy, vyl, axl, Lveh, L1, Weh, Wl, Wroad, logging): 
        
                """
                calculates the desired interaction lateral acceleration
        
                Parameters:
                    x: front position of the subjet vehicle
                    xl: front position of the leading vehicle 
                    vx: longitudinal speed of the subject vehicle 
                    vxl: longitudinal speed of the leading vehicle 
                    vy: lateral speed of the subject vehicle 
                    vyl: lateral speed of the leading vehicle 
                    axl: longitudinal acceleration of the leading vehicle 
                    Lveh: Length of subject vehicle 
                    L1: Length of the leader 
                    Weh: Width of the subject vehicle 
                    W1: Width of the leading vehicle (imo)
                    Wroad: width of the road 
        
                    dx: longitudinal distance =u[other vehicle]-u [m]
                    dy: lateral distance =v[other vehicle]-v [m]
                    sx: determined whether there is an overlap or not (0 for overlap, the gap between vehicles if not)
        
                Returns:
                    calc_acc_lat_int: desired lateral interaction acceleration [m/s^2] (including sign)
                
                """
        
                dx = xl - x 
                sx = max(0,dx)
                acc_cf_int = self.long_model.calc_acc_int(sx, vx, vxl, axl)
        
                dy = yl - y
                sign_dl = -1 if dy < 0 else 1
                Wavg = 0.5*(Weh+Wl)
        
                overlap = (abs(dy) < Wavg)
        
                alpha = -sign_dy*(abs(dy)/Wavg if (overlap) else exp(abs(dy)-Wavg)/self.s0y_lat)  # we have an confusion here to get solved 
        
                if overlap == True:
        
                    sylb_right = 0.5*Wroad - yl - 0.5*Wl; # right gap leader and road boundary 
                    sylb_left = Wroad - sylb_right - Wl # left gap leader and road boundary 
                    too_narrow_right = (sylb_right< Wveh + self.s0y_latb)
                    too_narrow_left = (sylb_left < Wveh + self.s0y_latb)
        
                    if not (too_narrow_right and too_narrow_left): 
        
                        if (too_narrow_right and (y>yl)): alpha = -1 
                        if (too_narrow_left and (y < yl)): alpha = 1
        
                v0_lat_int = -(self.sens_lat)*alpha*acc_cf_int
        
                if overlap == True: 
                    mult_dv_factor = 1
                else: 
                    mult_dv_factor = max(0,1-self.sens_dvy*sign_dy*(vyl-vy))
        
                acc_lat_int = v0_lat_int / self.tau_lat_OVM*mult_dv_factor: # this part is different from the orignal equation 
        
                acc_lat_int = max(-self.acc_lat_int_max, min(self.acc_laat_int_max, acc_lat_int))
        
                if logging:
                    print("MTM.calcAccLatInt:",
                          "x=", formd(x),
                          "dx=", formd(dx),
                          "y=", formd(y),
                          "dy=", formd(dy),
                          "vx=", formd(vx),
                          "vy=", formd(vy),
                          "accCFint=", formd(accCFint),
                          "\n                     alpha=", formd(alpha),
                          "mult_dv_factor=", formd(mult_dv_factor),
                          "v0LatInt=", formd(v0LatInt),
                          "accLatInt=", formd(accLatInt))
        
                return acc_lat_int


        ##############################################################
        ## Lateral and longitudinal boundaries 
        ##############################################################

        
        def alpha_long_b_fun(sy):
        
            """
            Calculate lateral attenuation factor for longitudinal boundary 
            
            Parameters:
                sy: abs(dy) - Wavg: calculated in calc_acc_long_int
            
            returns: lateral attenuation factor for longitudinal boundary 
                 
            """
        
            if sy > 0: 
                alpha = exp(-sy/self.s0y_b)
            else: 
                alpha = 1 
        
            return alpha 

        def alpha_lat_b_fun(sy): 
           
            """
            Calculate lateral attenuation factor for lateral boundary 
            
            Parameters:
                sy: abs(dy) - Wavg: calculated in calc_acc_long_int
            
            returns: lateral attenuation factor for lateral boundary 
                 
            """
        
            if sy > 0: 
                alpha = exp(-sy/self.s0y_lat_b)
            else: 
                alpha = 1 - sy/this.s0y_lat_b
        
            return alpha 


        def calc_acc_b(width_left, width_right, x, y, vx, vy, Wveh):
        
            """
            Calculate lateral and longitudianl boundary effect for both lateral and longitudinal acceleration 
            
            Parameters:
               width_left:  function pointer roadAxis-leftBd as a funct of arcLength u
               width_right: same for rightBd-roadAxis
               x = x position of the subejct 
               y = y position of the subject 
               vx = longitudinal speed of the subject   
               vy = lateral speed of the subject
               Wveh = width of the subject vehicle 
            
            returns: boundary effect for both lateral and longitudinal acceleration 
                 
            """    
            
            log = false 
        
            Tantic = self.antic_factor_b*(this.long_model.T)
            dTantic = 0 
        
            alpha_long_left_max = 0
            alpha_long_right_max = 0 
            alpha_lat_left_max = 0 
            alpha_lat_right_max = 0 
        
            v0y_b_left = 0 
            v0y_b_right = 0 
        
            # loop over spatial anticipations dx_antic=x+vx*TTC: find max interaction
        
            for j in range(self.nj):
        
                TTC = j*dTantic 
                weight = exp(-TTC/Tantic)
                sy_left = width_left(x+vx*TTC) + y - 0.5*Wveh  # y positive to right
                sy_right = width_right(x*vx*TTC) -y - 0.5*Wveh # y corresp to vehicle.v
        
                if (j > 0): 
        
                    v0y_b_left = max(v0y_b_left, -sy_left / TTC)
                    v0y_b_right = min(v0y_b_left, sy_right/TTC)
        
                alpha_long_left = self.alpha_long_b_fun(sy_left)*weight
                alpha_long_right = self.alpha_long_b_fun(sy_right)*weight
                alpha_lat_left = self.alpha_lat_b_fun(sy_left)*weight
                alpha_lat_right = self.alpha_lat_b_fun(sy_right)*weight
        
                
                alpha_long_left_max = max(alpha_long_left,alpha_long_left_max )
                alpha_long_right_max =  max(alpha_long_right,alpha_long_right_max )
                alpha_lat_left_max = max(alpha_lat_left,alpha_lat_left_max )
                alpha_lat_right_max =  max(alpha_lat_right, alpha_lat_right_max)
                
                v0y = v0y_b_left if (abs(v0y_b_left) > abs(v0y_b_right)) else v0y_b_right 
                
                acc_long_b = self.acc_long_b_ref*(-alpha_long_left_max -alpha_long_right_max)
                acc_lat_b = self.acc_lat_b_ref*(alpha_lat_left_max - alpha_lat_right_max)
        
                acc_long_b *= vx/v0max
                acc_lat_b *= (0.2 + 0.8*vx/v0max)
        
                acc_lat_b_restr = max(-self.acc_lat_b_max, min(acc_lat_b_max, acc_lat_b)) # rarely in effect#
        
                # return is changed from the original js code, since the main objective is to get acc long b and acc lat b 
                return  acc_long_b, acc_lat_b
                
